In [ ]:
# Download the datatset from Kaggle.
!kaggle datasets download -p datasets/oxford-iiit-pet -d tanlikesmath/the-oxfordiiit-pet-dataset --unzip

In [ ]:
import tensorflow as tf
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
cats = ['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair', 'Egyptian_Mau', 
        'Maine_Coon', 'Persian', 'Ragdoll', 'Russian_Blue', 'Siamese', 'Sphynx']

class_names = ['cat', 'dog']

data = []
for img in glob.glob('datasets/oxford-iiit-pet/images/*.jpg'):
    label = 'cat' if any(cat in img for cat in cats) else 'dog'
    data.append((img, label))

print(f'There are {len(data)} images')

In [ ]:
# Shuffle the data before partitioning
np.random.shuffle(data)

# Split the data into train, validation and test sets
train, val, test = np.split(data, [int(len(data) * 0.7), int(len(data) * 0.8)])

train_df = pd.DataFrame({'image':train[:,0], 'label':train[:,1]})
val_df = pd.DataFrame({'image':val[:,0], 'label':val[:,1]})
test_df = pd.DataFrame({'image':test[:,0], 'label':test[:,1]})

print(f'There are {len(train_df)} images for training')
print(f'There are {len(val_df)} images for validation')
print(f'There are {len(test_df)} images for testing')

In [ ]:
# Give the CNN 32 images at a time. The lower the batch size, the better the model will learn,
# however, the training process will be longer.
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224

# Create the ImageDataGenerator object and rescale the images
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

# Convert them into a dataset to be split into batches, shuffled and resized
train_dataset = image_generator.flow_from_dataframe(
    dataframe=train_df,
    class_mode='binary',
    x_col='image',
    y_col='label',
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    target_size=(IMG_HEIGHT,IMG_WIDTH)
)

val_dataset = image_generator.flow_from_dataframe(
    dataframe=val_df,
    class_mode='binary',
    x_col='image',
    y_col='label',
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    target_size=(IMG_HEIGHT,IMG_WIDTH)
)

test_dataset = image_generator.flow_from_dataframe(
    dataframe=test_df,
    class_mode='binary',
    x_col='image',
    y_col='label',
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
    target_size=(IMG_HEIGHT,IMG_WIDTH)
)

train_images, train_labels = next(iter(train_dataset))

print(f'Batch shape: {train_images.shape}')
print(f'Label shape: {train_labels.shape}')

In [ ]:
# Verify the data by plotting the first few images in the dataset
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([]), plt.yticks([])
    plt.imshow(train_images[i])
    plt.xlabel(class_names[int(train_labels[i])])
plt.show()

In [ ]:
# Build the convolutional neural network, specifying each of the layers
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape = [IMG_HEIGHT, IMG_WIDTH, 3]),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
# Specify the algorithm for backpropagation, the loss function and a performace metric
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])
# Stop training early if the validation loss is constant or increasing for more than 3 epochs
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# Train the network
history = model.fit(train_dataset, epochs=15, validation_data=val_dataset, callbacks=callback)

In [ ]:
def plot_metric(history, metric='loss'):
    plt.title(metric.capitalize())
    plt.plot(history.history[metric])
    plt.plot(history.history[f'val_{metric}'])
    plt.xlabel('Epoch'), plt.ylabel(metric.capitalize())
    plt.legend(['Training', 'Validation'])
    plt.show()

# Evaluate the network
plot_metric(history, 'loss')
plot_metric(history, 'accuracy')

# Test the network on unseen data
loss, acc = model.evaluate(test_dataset)

In [ ]:
def process_image(url):
    image = plt.imread(tf.keras.utils.get_file(origin=url))
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH]) / 255
    image = np.expand_dims(image, 0)
    return image

image = process_image('https://upload.wikimedia.org/wikipedia/commons/d/d5/Retriever_in_water.jpg')
predictions = model.predict(image)

plt.xticks([]), plt.yticks([])
plt.xlabel(f'{class_names[round(predictions[0, 0])]} ({np.max(predictions):.2f})')
plt.imshow(image[0,:,:,:])

In [ ]:
# Save the model
model.save('saved/image-classifier-oxford-iiit-pet.h5')